In [1]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ROOT
from pathlib import Path
import mplhep as hep

hep.style.use(hep.style.ATLAS)

import re
import joblib
import logging
from utils.utils import check_inputpath, check_outputpath, logging_setup

Welcome to JupyROOT 6.28/00


In [2]:
workdir = Path("../../")
assert workdir.resolve().name == "ssWWWZjj"

# Comapre GM 450GeV 

## Cutflow

In [ ]:
# This is for the un-decorated samples where a cutflow is stored in the ROOT file
# GMH5_m450_ROOT = ROOT.TFile.Open(GM_sig_m450_path.as_posix())
# GMH5_m450_ROOT.ls()

# c1 = ROOT.TCanvas("c1", "c1", 800, 600)
# GMH5_m450_ROOT.Get("event_CutFlow").Draw()
# c1.Draw()

## Check GM H5 mass = 450 GeV

In [3]:
GM_sig_m450_path = (
    workdir
    / "samples/Run2_WZ_samples/decorated/GM_sig/resonance.450770_MGaMcAtNloPy8EG_A14NNPDF23LO_vbfGM_sH05_H5pWZ_lvll_m450_ntuples.root"
)
GM_sig_m450_path = Path(GM_sig_m450_path)

In [ ]:
GM_sig_m450 = uproot.open(GM_sig_m450_path)

In [ ]:
GM_sig_m450.keys()

In [ ]:
nominal_branchs = GM_sig_m450["nominal"].keys()[0:93] + ["Ht", "pSignal_GM"]
GM_sig_m450_nominal_branches = GM_sig_m450["nominal"].arrays(nominal_branchs, library="pd")

In [ ]:
for branch in nominal_branchs:
    print(branch)

In [ ]:
##### Add cuts accrording to the table 1
cut_GM_sig_m450 = GM_sig_m450_nominal_branches.copy()
training_cutflow = {}
# WZInclusive
cut_GM_sig_m450 = cut_GM_sig_m450.loc[cut_GM_sig_m450["WZInclusive"] == 1]
training_cutflow["Cut WZInclusive"] = len(cut_GM_sig_m450)
print(f"Cut WZInclusive: {len(cut_GM_sig_m450)}")

# N_j >= 2
cut_GM_sig_m450 = cut_GM_sig_m450.loc[cut_GM_sig_m450["Njets"] >= 2]
training_cutflow["Cut Njets>=2"] = len(cut_GM_sig_m450)
print(f"Cut Njets>=2: {len(cut_GM_sig_m450)}")

# b-jet veto
cut_GM_sig_m450 = cut_GM_sig_m450.loc[cut_GM_sig_m450["NBjets"] == 0]
training_cutflow["Cut NBjets==0"] = len(cut_GM_sig_m450)
print(f"Cut NBjets==0: {len(cut_GM_sig_m450)}")

# M_jj > 100 GeV
cut_GM_sig_m450 = cut_GM_sig_m450.loc[cut_GM_sig_m450["M_jj"] >= 100]
training_cutflow["Cut M_jj>100GeV"] = len(cut_GM_sig_m450)
print(f"Cut M_jj>100GeV: {len(cut_GM_sig_m450)}")

training_cutflow = pd.DataFrame(
    {
        "training_cutflow": training_cutflow.keys(),
        "cut_yield": training_cutflow.values(),
    }
)

In [ ]:
training_cutflow

# Check all H5 masses

In [ ]:
##### Reproduce table 11

In [4]:
input_path = workdir / "samples/Run2_WZ_samples/decorated/"
input_path = Path(input_path)
check_inputpath(input_path)

GM_sig_path = input_path / "GM_sig"

In [5]:
def apply_training_cut(GM_sig_m_nominal_branches):
    cut_GM_sig_m = GM_sig_m_nominal_branches.copy()
    training_cutflow = {}
    # WZInclusive
    cut_GM_sig_m = cut_GM_sig_m.loc[cut_GM_sig_m["WZInclusive"] == 1]
    training_cutflow["Cut WZInclusive"] = len(cut_GM_sig_m)

    # N_j >= 2
    cut_GM_sig_m = cut_GM_sig_m.loc[cut_GM_sig_m["Njets"] >= 2]
    training_cutflow["Cut Njets>=2"] = len(cut_GM_sig_m)

    # b-jet veto
    cut_GM_sig_m = cut_GM_sig_m.loc[cut_GM_sig_m["NBjets"] == 0]
    training_cutflow["Cut NBjets==0"] = len(cut_GM_sig_m)

    # M_jj > 100 GeV
    cut_GM_sig_m = cut_GM_sig_m.loc[cut_GM_sig_m["M_jj"] >= 100]
    training_cutflow["Cut M_jj>100GeV"] = len(cut_GM_sig_m)

    training_cutflow = pd.DataFrame(
        {
            "training_cutflow": training_cutflow.keys(),
            "cut_yield": training_cutflow.values(),
        }
    )

    raw_event_yield = len(cut_GM_sig_m)
    normalized_event_yield = cut_GM_sig_m["WeightNormalized"].sum()

    return (
        cut_GM_sig_m,
        training_cutflow,
        raw_event_yield,
        normalized_event_yield,
    )

In [6]:
training_input_yield = {
    "mass": [],
    "raw_event_yield": [],
    "normalized_event_yield": [],
}

re_mass_pattern = r"m(\d+)(?=[_ntuples,_lepfilt])"

merged_sig = []

for GM_sig_file in sorted(GM_sig_path.glob("*.root")):
    GM_sig_file_name = GM_sig_file.name
    match = re.search(re_mass_pattern, GM_sig_file_name)
    if match:
        mass = match.group(1)
        mass = int(mass)
    else:
        raise ValueError("No mass found in file name")

    GM_sig_file = uproot.open(GM_sig_file)
    nominal_branchs = GM_sig_file["nominal"].keys()[0:93] + [
        "Ht",
        "pSignal_GM",
    ]
    GM_sig_m_nominal_branches = GM_sig_file["nominal"].arrays(nominal_branchs, library="pd")

    (
        cut_GM_sig_m,
        training_cutflow,
        raw_event_yield,
        normalized_event_yield,
    ) = apply_training_cut(GM_sig_m_nominal_branches)

    cut_GM_sig_m["file_identifer"] = f"signal_m{mass}"
    cut_GM_sig_m["target"] = 1
    merged_sig.append(cut_GM_sig_m)
    training_input_yield["mass"].append(mass)
    training_input_yield["raw_event_yield"].append(raw_event_yield)
    training_input_yield["normalized_event_yield"].append(normalized_event_yield)

In [7]:
training_input_yield = pd.DataFrame(training_input_yield)
merged_sig = pd.concat(merged_sig)

In [11]:
training_input_yield.loc["Total"] = training_input_yield.sum()

In [12]:
training_input_yield

,mass,raw_event_yield,normalized_event_yield
0,200.0,12153.0,152.474777
1,250.0,27175.0,142.906448
2,300.0,30644.0,137.715607
3,350.0,37346.0,128.195175
4,400.0,42109.0,115.105957
5,450.0,40272.0,91.894867
6,500.0,46936.0,81.789330
7,225.0,23628.0,146.715729
8,275.0,29752.0,145.312027
9,325.0,35208.0,128.033997


In [8]:
joblib.dump(merged_sig, "merged_decorated_sig.pkl")

['merged_decorated_sig.pkl']

In [9]:
merged_sig.head()

,Yields,isMC,Channel,Year,NormSF,WeightSign,WeightNormalized,Weight,M_WZ,M_123,...,Lep1WeightW,Lep2WeightW,Lep3WeightW,Lep1Level,Lep2Level,Lep3Level,Ht,pSignal_GM,file_identifer,target
3,0,450765,3,1516,0.783916,0.0,-0.079258,-0.101105,187.706635,144.789856,...,0.969956,0.990138,0.995931,123,123,123,508.242188,0.781384,signal_m200,1
11,0,450765,2,1516,0.783916,0.0,-0.053379,-0.068093,270.459198,156.205460,...,0.996840,0.998581,0.978706,123,123,123,295.828552,0.887640,signal_m200,1
14,0,450765,3,1516,0.783916,0.0,0.042182,0.053809,196.065414,165.009293,...,0.971638,0.966696,0.998496,123,123,123,274.689301,0.922267,signal_m200,1
16,0,450765,2,1516,0.783916,0.0,-0.066834,-0.085256,200.703903,140.544388,...,0.995598,0.981509,0.902356,123,123,123,288.953735,0.939477,signal_m200,1
22,0,450765,1,1516,0.783916,0.0,0.072323,0.092259,221.749603,144.889526,...,0.990660,0.975106,0.995559,123,123,123,261.893311,0.621475,signal_m200,1


# Merge Bkg

In [13]:
bkg_path = input_path / "bkg"
check_inputpath(bkg_path)

PosixPath('../../samples/Run2_WZ_samples/decorated/bkg')

In [14]:
sorted(bkg_path.iterdir())

[PosixPath('../../samples/Run2_WZ_samples/decorated/bkg/resonance.364253_Sherpa_222_NNPDF30NNLO_lllv_ntuples.root'),
 PosixPath('../../samples/Run2_WZ_samples/decorated/bkg/resonance.364739_MGPy8EG_NNPDF30NLO_A14NNPDF23LO_lvlljjEW6_OFMinus_ntuples.root'),
 PosixPath('../../samples/Run2_WZ_samples/decorated/bkg/resonance.364740_MGPy8EG_NNPDF30NLO_A14NNPDF23LO_lvlljjEW6_OFPlus_ntuples.root'),
 PosixPath('../../samples/Run2_WZ_samples/decorated/bkg/resonance.364741_MGPy8EG_NNPDF30NLO_A14NNPDF23LO_lvlljjEW6_SFMinus_ntuples.root'),
 PosixPath('../../samples/Run2_WZ_samples/decorated/bkg/resonance.364742_MGPy8EG_NNPDF30NLO_A14NNPDF23LO_lvlljjEW6_SFPlus_ntuples.root')]

In [15]:
training_input_yield_bkg = {
    "name": [],
    "raw_event_yield": [],
    "normalized_event_yield": [],
}

merged_bkg = []

for bkg_file in sorted(bkg_path.glob("*.root")):
    bkg_file_name = bkg_file.name
    bkg_file = uproot.open(bkg_file)
    nominal_branchs = bkg_file["nominal"].keys()[0:93] + ["Ht", "pSignal_GM"]
    bkg_nominal_branches = bkg_file["nominal"].arrays(nominal_branchs, library="pd")

    (
        cut_bkg,
        training_cutflow,
        raw_event_yield,
        normalized_event_yield,
    ) = apply_training_cut(bkg_nominal_branches)

    cut_bkg["file_identifer"] = bkg_file_name
    cut_bkg["target"] = 0
    merged_bkg.append(cut_bkg)

    training_input_yield_bkg["name"].append(bkg_file_name)
    training_input_yield_bkg["raw_event_yield"].append(raw_event_yield)
    training_input_yield_bkg["normalized_event_yield"].append(normalized_event_yield)

In [16]:
training_input_yield_bkg = pd.DataFrame(training_input_yield_bkg)
training_input_yield_bkg.loc["Total"] = training_input_yield_bkg.sum()

In [17]:
training_input_yield_bkg

,name,raw_event_yield,normalized_event_yield
0,resonance.364253_Sherpa_222_NNPDF30NNLO_lllv_n...,816454,2973.739502
1,resonance.364739_MGPy8EG_NNPDF30NLO_A14NNPDF23...,25815,40.750809
2,resonance.364740_MGPy8EG_NNPDF30NLO_A14NNPDF23...,35364,64.986855
3,resonance.364741_MGPy8EG_NNPDF30NLO_A14NNPDF23...,24430,38.421360
4,resonance.364742_MGPy8EG_NNPDF30NLO_A14NNPDF23...,30272,61.633919
Total,resonance.364253_Sherpa_222_NNPDF30NNLO_lllv_n...,932335,3179.532471


In [18]:
merged_bkg = pd.concat(merged_bkg)

In [ ]:
np.sum(merged_bkg["WZVBSSR"])

In [19]:
joblib.dump(merged_bkg, "merged_decorated_bkg.pkl")

['merged_decorated_bkg.pkl']

# Signal vs Bkg 

In [ ]:
plots_path = Path("./plots_sigvsbkg")
check_outputpath(plots_path)

In [ ]:
merged_sig.columns

In [ ]:
np.sum(merged_sig["WZVBSSR"])

In [ ]:
def compare_sigvsbkg(feature, merged_sig, merged_bkg):
    feature_min = np.min([np.min(merged_sig[feature]), np.min(merged_bkg[feature])])
    feature_max = np.max([np.max(merged_sig[feature]), np.max(merged_bkg[feature])])
    if feature == "M_jj":
        feature_min = 0
        feature_max = 5000
    elif feature == "Met":
        feature_min = 0
        feature_max = 400
    elif feature == "Jet1Pt":
        feature_min = 0
        feature_max = 600
    elif feature == "Jet2Pt":
        feature_min = 0
        feature_max = 300
    elif feature == "Ht":
        feature_min = 0
        feature_max = 1500

    feature_bins = np.linspace(feature_min, feature_max, 101)
    sig_bin_contents, sig_bin_edges = np.histogram(
        merged_sig[feature],
        bins=feature_bins,
        density=True,
        weights=merged_sig["WeightNormalized"],
    )
    bkg_bin_contents, bkg_bin_edges = np.histogram(
        merged_bkg[feature],
        bins=feature_bins,
        density=True,
        weights=merged_bkg["WeightNormalized"],
    )

    bin_content_max = np.max([np.max(sig_bin_contents), np.max(bkg_bin_contents)])
    fig, ax = plt.subplots()
    ax.set_xlim(feature_min, feature_max)
    ax.set_ylim(0, bin_content_max * 1.3)

    hep.histplot(sig_bin_contents, sig_bin_edges, label="Signal", ax=ax, color="red")
    hep.histplot(
        bkg_bin_contents,
        bkg_bin_edges,
        label="Background",
        ax=ax,
        color="blue",
    )

    hep.atlas.label("Internal", data=False)
    hep.atlas.set_ylabel("Normalized to unity")
    hep.atlas.set_xlabel(feature)
    ax.legend()

    fig.savefig(plots_path / f"{feature}.png")

In [ ]:
features = [
    "Deta_jj",
    "Dphi_jj",
    "M_jj",
    "Met",
    "ZetaLep",
    "Jet1Pt",
    "Jet1Eta",
    "Jet2Pt",
    "Jet2Eta",
    "Ht",
    "Eta_W",
    "Eta_Z",
    "Lep1Eta",
    "Lep2Eta",
    "Lep3Eta",
    "pSignal_GM",
]

for feature in features:
    compare_sigvsbkg(feature, merged_sig, merged_bkg)